In [ ]:
import dataset_loader
import matplotlib.pyplot as plt
import midi
import numpy as np
import similarity

In [ ]:
DATASET_ROOT="../../sabana/dataset/asap-dataset"

SLICE_DURATION = 5 # sec
EXPANSION_RATE = 1.5
FRAME_PER_SECOND = 20 # Hz
NUM_SAMPLES = 1
SETTLING_FRAME = 8
COMPENSATION_FRAME = 0

In [ ]:
gen = dataset_loader.spawn(dataset_root=DATASET_ROOT, 
                           slice_duration=SLICE_DURATION,
                           expansion_rate=EXPANSION_RATE,
                           frame_per_second=FRAME_PER_SECOND,
                           shuffle=True)

In [ ]:
score, perf, (head, tail) = gen.__next__()

In [ ]:
def plot_midi_matrix(matrix: np.ndarray, title: str, head: int = None, tail: int = None):
    plt.figure(figsize=(16, 3))
    plt.title(title)
    plt.imshow(matrix, cmap="gray", aspect="auto", origin="lower", interpolation="nearest")
    if head:
        plt.plot([head, head], [0, 128])
    if tail:
        plt.plot([tail, tail], [0, 128])
    plt.ylabel("MIDI Key")
    plt.xlabel("Frame")
    plt.show()
    plt.clf()
    return None

In [ ]:
def process_midi_matrix(midi_matrix, settling_frame):
    prev_pressed = [False] * midi.NUM_MIDI_KEYS
    midi_matrix = np.reshape(midi_matrix, [midi.NUM_MIDI_KEYS, -1]).T
    for frame_idx in range(len(midi_matrix)):
        for midi_key in range(midi.NUM_MIDI_KEYS):
            velocity = 0
            if midi_matrix[frame_idx, midi_key] <= 0:
                prev_pressed[midi_key] = False
            elif frame_idx > 0:
                prev_velocity = midi_matrix[frame_idx - 1, midi_key]
                if prev_velocity > 0:
                    velocity = prev_velocity - 1
                elif not prev_pressed[midi_key]:
                    velocity = settling_frame - 1
                prev_pressed[midi_key] = True
            else:
                velocity = settling_frame - 1
                prev_pressed[midi_key] = True
            midi_matrix[frame_idx, midi_key] = velocity
    return midi_matrix

In [ ]:
score, perf, (head, tail) = gen.__next__()

In [ ]:
plot_midi_matrix(score, "score")
score_ = process_midi_matrix(score, SETTLING_FRAME).T
plot_midi_matrix(score_, "score")

plot_midi_matrix(perf, "perf")
perf_ = process_midi_matrix(perf, SETTLING_FRAME).T
plot_midi_matrix(perf_, "perf")

In [ ]:
for i in range(100):
    print("=============================================================")
    score, perf, (head, tail) = gen.__next__()
    plot_midi_matrix(score, "score")
    plot_midi_matrix(perf, "perf", head, tail)

    score_ = process_midi_matrix(score, SETTLING_FRAME).T
    plot_midi_matrix(score_, "score")
    perf_ = process_midi_matrix(perf, SETTLING_FRAME).T
    plot_midi_matrix(perf_, "perf", head, tail)

In [ ]:
score__ = midi.MIDIUnitSequenceList.from_midi_matrix(score,SETTLING_FRAME)
perf__ = midi.MIDIUnitSequenceList.from_midi_matrix(perf,SETTLING_FRAME)

In [ ]:
score_sequence = score__.to_representative_unit_sequence()
perf_sequence = perf__.to_representative_unit_sequence()

In [ ]:
plot_midi_matrix(score_, "perf")
print(score__)
print()
print(score_sequence)

In [ ]:
plot_midi_matrix(perf_, "perf")
print(perf__)
print()
print(perf_sequence)